# Dead reckoning for a unicycle model
The state of the unicycle model is
$$ \zeta = \begin{bmatrix} \theta\\x\\y \end{bmatrix}, $$
and its kinematics is given by
$$ \dot{\zeta} = \begin{bmatrix} \dot{\theta} \\ \dot{x} \\ \dot{y} \end{bmatrix} = \begin{bmatrix} \omega \\ v\cos\theta \\ v\sin\theta \end{bmatrix}, $$
where $v$ is the linear velocity and $\omega$ is the angular velocity, both of which are considered input signals to the model.

If we have an estimate of the state at some point in time $t_k$, $\hat{\zeta}(t_k) = \hat{\zeta}_k$, we can estimate the state at some (short) time later by integrating the kinematics equation, given knowledge of the input signals. It is common to use Euler's approximation to the derivative 
$$ \dot{\zeta}_k \approx \frac{\zeta_{k+1} - \zeta_k}{\Delta t}, $$
which gives
$$ \frac{\hat{\zeta}_{k+1} - \hat{\zeta}_k}{\Delta t} =  \begin{bmatrix} \omega_k \\ v_k\cos\theta_k \\ v_k\sin\theta_k \end{bmatrix}, $$
leading to 
$$ \hat{\zeta}_{k+1} = \hat{\zeta}_k + \Delta t \begin{bmatrix} \omega_k \\ v_k\cos\theta_k \\ v_k\sin\theta_k \end{bmatrix}. $$

## Example: circular motion
The robot is moving along a circular path with constant linear velocity $v$=1 m/s, and constant angular velocity $\omega$=1 rad/s. Clearly, the radius of the circle is $r$=1 m. At time $t=0$ the robot starts at the origin with linear velocity in the direction of the x-axis. The true state is thus given by
$$\zeta(t) = \begin{bmatrix} \theta(t)\\x(t)\\y(t) \end{bmatrix} = \begin{bmatrix} t \\ \sin(t) \\ 1 - \cos(t) \end{bmatrix}. $$ 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# Plotting the path
tv = np.linspace(0,2*np.pi, 100)
plt.figure()
plt.plot(np.sin(tv), 1-np.cos(tv))

## Implement dead reckoning
Implement dead reckoning and compare the resulting path with the true path.

In [ ]:
zeta0 = np.array([0.0, 0.0, 0.0]) # The initial state
zetahat = [] # The estimated states
tt = [] # The time vector
dt = 0.3
t = 0
zeta_k = zeta0
def v(t): # The linear velocity input signal
    return 1.0
def w(t): # The angular velocity input signal
    return 1.0
while (t < 2*np.pi): # Do one complete revolution
    zetahat.append(zeta_k.copy())
    tt.append(t)
    th_k = zeta_k[0] # The heading angle
    w_k = w(t) # The angular velocity
    v_k = v(t) # The linear velocity 
    #-----------------------------------------
    # YOUR CODE HERE
    #-----------------------------------------

In [ ]:
# Plot the results
zetahat = np.array(zetahat)
plt.figure()
plt.plot(np.sin(tv), 1-np.cos(tv), 'b', zetahat[:,1], zetahat[:,2], 'ro')
plt.show()

In [ ]:
plt.figure()
plt.plot(tv, tv, 'b', tt, zetahat[:,0], 'ro')